# RAMSES Fundamentals

## 1. Prerequisites

**Note: I (Marion) copied and summarised the 1.1 and 1.2 below under the "Setting up" section of the SNO tuto's doc.**

### 1.1. On your personal computer

### Compilers

You will need a **fortran compiler** to compile RAMSES, along with some version of **MPI** if you have access to multiple cores and want to run RAMSES in parallel. You can refer to the **Instructions to get started on your laptop** section for some information about how to install those.

We recommend installing recent versions of the GNU fortran compiler `gfortran` [(link)](https://gcc.gnu.org/fortran/) and `openmpi` [(link)](https://www.open-mpi.org/).


### Python

The tutorials require running `python3` in a `Jupyter notebook`. 

We will use a the following python packages: 
```
numpy, astropy, matplotlib
```

Depending on the tutorials, different analysis tools will also be used, including : 
```
yt, yt_astro_analysis, colossus, osiris, ... 
```

### 1.2. On a supercomputer

Here, fortran compilers and MPI libraries will generally be installed and you need to find out how to use them. Often, this is done by loading modules ... **GIVE EXAMPLES** 

## 2. First steps with RAMSES

RAMSES [(Teyssier 2002)](https://ui.adsabs.harvard.edu/abs/2002A%26A...385..337T/abstract) is a popular code used to produce astrophysical simulations, using the adaptive mesh refinement (AMR) numerical technique, together with state-of-the art star formation and feedback models. Each directory contains a set of files with a given common purpose. For example, amr/ contains all Fortran 90 routines dealing with the AMR grid management and MPI communications, while hydro/ obviously contains all Fortran 90 routines dealing with hydrodynamics. RAMSES is written in Fortran 90, which can be used on massively parallel architectures thanks to the MPI library (MPI stands for Message Passing Interface). Therefore, before producing RAMSES simulations, we need to compile the RAMSES code, as explained below.

In order to **obtain and compile the code**, you can follow the first steps of the documentation at [this link](https://ramses-organisation.readthedocs.io/en/latest/wiki/Start.html), summarised (and updated) below.

### 2.1 Downloading the code

To get the RAMSES code, open a terminal, and clone the code with the following command:

```bash
git clone https://github.com/ramses-organisation/ramses
```

### 2.2 Compiling the code

 The first directory you are interested in is the bin/ directory, in which the code will be compiled thanks to the set of instructions written in the `Makefile`.

```bash
cd ramses/bin
cat Makefile
```

The first lines of the Makefile correspond to the so-called *Compilation Time Parameters*. They are used to:

- set the type of compiler to be used, the name of the RAMSES executable

- define the number of dimensions and the total number of variables needed for the simulation

- enable or disable the compilation of some of the physical modules, of the MPI routines 

Other preprocessor directives are then defined below, in the variable `DEFINES`.

Based on the choice of compiler (`COMPILER=GNU` or `INTEL`), Fortran compiler options and directives are defined. The `F90` variable invokes the Fortran compiler (also depends on the use of MPI), while `FFLAGS` contains other compilation flags and preprocessor directives (also depends on the use of debug options). While both `F90` and `FFLAGS` can be set up using default values, one might need or want to edit them according to the compiler, the machine, or the simulation. In this case, you can manually edit `F90` and `FFLAGS` in the Makefile.

Finally, the rest of the Makefile contains the list and path of the files to be compiled.

Once the Makefile is ready, you can (re)compile the code:
```bash
make clean
make
```

### 2.3 The namelist

Maybe quickly explain what it is here?

## 3. Exercise 1